## Урок 5. Part-of-Speech разметка, NER, извлечение отношений


### Тема “POS-tagger и NER”

### Задание 1. Написать теггер на данных с русским языком
проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации
написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
сравнить все реализованные методы сделать выводы
 


In [1]:
!pip install pyconll

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install ipymarkup

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26119 sha256=dcad6df1fe5f642a26b91c2c70c7180ea500190b27fdb37e5da0ede324afae49
  Stored in directory: /root/.cache/pip/wheels/16/85/bd/1001cbb46dcfb71c2001cd7401c6fb250392f22a81ce3722f7
Successfully built intervaltree
  Attempting uninstall: intervaltree
    Found existing installation: intervaltree 2.1.0
    Uninstalling intervaltree-2.1.0:
      Successfully uninstalled intervaltree-2.1.0


In [3]:
!pip install corus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 2.2 MB/s 


In [4]:
!pip install sklearn_crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 965 kB 19.1 MB/s 


In [8]:
#nltk.download() 
import nltk
from nltk.tokenize import word_tokenize
import matplotlib
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import pyconll
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report

import os
import corus
!pip install -U spacy
!python -m spacy info
!python -m spacy download ru_core_news_sm
import spacy
from spacy import displacy
import ru_core_news_sm
!pip install slovnet
from navec import Navec
from slovnet import NER
from ipymarkup import show_span_ascii_markup as show_markup
from razdel import tokenize
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.metrics import f1_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-07-15 20:13:50.687633: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
/usr/local/lib/python3.7/dist-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_core_web_sm' (3.3.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)

============================== Info about spaCy ==============================

spaCy version    3.4.0                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.13                        


In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
full_train = pyconll.load_from_file('/content/ru_syntagrus-ud-train-a.conllu')
full_test = pyconll.load_from_file('/content/ru_syntagrus-ud-dev.conllu')

In [18]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [19]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [20]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 194
Наибольшая длина токена 31


In [21]:
len(fdata_train), len(fdata_test), len(fdata_sent_test)

(24516, 8906, 8906)

In [22]:
fdata_train[:5]

[[('Анкета', 'NOUN'), ('.', 'PUNCT')],
 [('Начальник', 'NOUN'),
  ('областного', 'ADJ'),
  ('управления', 'NOUN'),
  ('связи', 'NOUN'),
  ('Семен', 'PROPN'),
  ('Еремеевич', 'PROPN'),
  ('был', 'AUX'),
  ('человек', 'NOUN'),
  ('простой', 'ADJ'),
  (',', 'PUNCT'),
  ('приходил', 'VERB'),
  ('на', 'ADP'),
  ('работу', 'NOUN'),
  ('всегда', 'ADV'),
  ('вовремя', 'ADV'),
  (',', 'PUNCT'),
  ('здоровался', 'VERB'),
  ('с', 'ADP'),
  ('секретаршей', 'NOUN'),
  ('за', 'ADP'),
  ('руку', 'NOUN'),
  ('и', 'CCONJ'),
  ('иногда', 'ADV'),
  ('даже', 'PART'),
  ('писал', 'VERB'),
  ('в', 'ADP'),
  ('стенгазету', 'NOUN'),
  ('заметки', 'NOUN'),
  ('под', 'ADP'),
  ('псевдонимом', 'NOUN'),
  ('"', 'PUNCT'),
  ('Муха', 'NOUN'),
  ('"', 'PUNCT'),
  ('.', 'PUNCT')],
 [('В', 'ADP'),
  ('приемной', 'NOUN'),
  ('его', 'PRON'),
  ('с', 'ADP'),
  ('утра', 'NOUN'),
  ('ожидали', 'VERB'),
  ('посетители', 'NOUN'),
  (',', 'PUNCT'),
  ('-', 'PUNCT'),
  ('кое-кто', 'PRON'),
  ('с', 'ADP'),
  ('важными', 'ADJ'),

In [23]:
unigram_tagger = UnigramTagger(fdata_train)
accuracy_U = unigram_tagger.evaluate(fdata_test)
display(accuracy_U)

0.823732013802982

In [24]:
bigram_tagger = BigramTagger(fdata_train)
accuracy_B = bigram_tagger.evaluate(fdata_test)
display(accuracy_B)

0.6093886320724006

In [25]:
trigram_tagger = TrigramTagger(fdata_train)
accuracy_T = trigram_tagger.evaluate(fdata_test)
display(accuracy_T)

0.1778631421316492

In [26]:
list_1 = [UnigramTagger, BigramTagger]

list_2 = [UnigramTagger, TrigramTagger]

list_3 = [BigramTagger, TrigramTagger]

list_4 = [UnigramTagger, BigramTagger, TrigramTagger]

accuracy_N = []

def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff

for list_N in [list_1, list_2, list_3, list_4]:
    backoff = DefaultTagger('NN') 
    tag = backoff_tagger(fdata_train,  
                         list_N,  
                         backoff = backoff) 

    accuracy_N.append(tag.evaluate(fdata_test))

accuracy_N

[0.8275343446838986, 0.8273650628296113, 0.1750309264926102, 0.827905462595221]

In [27]:
import pandas as pd

In [28]:
result = pd.DataFrame({'Tagger': ['Unigram', 'Bigram', 'Trigram', 'Unigram + Bigram', 'UnigramTagger + TrigramTagger', 'BigramTagger + TrigramTagger', 'UnigramTagger+BigramTagger+TrigramTagger'], 'Accuracy' : [accuracy_U, accuracy_B, accuracy_T] + accuracy_N})
result.sort_values('Accuracy', ascending=False)

,Tagger,Accuracy
6,UnigramTagger+BigramTagger+TrigramTagger,0.827905
3,Unigram + Bigram,0.827534
4,UnigramTagger + TrigramTagger,0.827365
0,Unigram,0.823732
1,Bigram,0.609389
2,Trigram,0.177863
5,BigramTagger + TrigramTagger,0.175031


In [29]:
train_tok = [] 
train_label = [] 
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [30]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [31]:
vectorizers = [CountVectorizer(ngram_range=(1, 3), analyzer='char'), 
               TfidfVectorizer(ngram_range=(1, 3), analyzer='char'), 
               HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=1000)] 
vectorizers_word = [CountVectorizer(ngram_range=(1, 3), analyzer='word'), 
               TfidfVectorizer(ngram_range=(1, 3), analyzer='word'), 
               HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=1000)] 
n_features = [2000, 3000, 5000, 10000]
vectorizers_hash = [HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=feat) for feat in n_features]
vectorizers_hash_word = [HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=feat) for feat in n_features]
f1_scores = []
accuracy_scores = []

for vectorizer in vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word:
    X_train = vectorizer.fit_transform(train_tok)
    X_test = vectorizer.transform(test_tok[:115000])
    
    lr = LogisticRegression(random_state=0, max_iter=100)
    lr.fit(X_train, train_enc_labels)
    pred = lr.predict(X_test)
    f1 = f1_score(test_enc_labels[:115000], pred, average='weighted')
    f1_scores.append(f1)
    acc = accuracy_score(test_enc_labels[:115000], pred)
    accuracy_scores.append(acc)
    
    print(vectorizer)
    print(classification_report(test_enc_labels[:115000], pred, target_names=le.classes_))

CountVectorizer(analyzer='char', ngram_range=(1, 3))
              precision    recall  f1-score   support

         ADJ       0.92      0.91      0.92     11247
         ADP       0.98      1.00      0.99     10255
         ADV       0.92      0.90      0.91      5986
         AUX       0.81      0.97      0.88      1058
       CCONJ       0.88      0.98      0.93      4276
         DET       0.88      0.75      0.81      2978
        INTJ       0.33      0.36      0.35        11
        NOUN       0.92      0.95      0.94     27241
      NO_TAG       1.00      1.00      1.00       197
         NUM       0.86      0.90      0.88      1436
        PART       0.95      0.78      0.86      3762
        PRON       0.83      0.89      0.86      5346
       PROPN       0.79      0.59      0.67      4315
       PUNCT       1.00      1.00      1.00     21941
       SCONJ       0.81      0.91      0.86      2176
         SYM       1.00      0.68      0.81        53
        VERB       0.94     

In [32]:
result_model = pd.DataFrame({'Vectorizer': vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word,
                            'f1_score': f1_scores})
result_model.sort_values('f1_score', ascending=False)

,Vectorizer,f1_score
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.927826
1,"TfidfVectorizer(analyzer='char', ngram_range=(...",0.921158
9,"HashingVectorizer(analyzer='char', n_features=...",0.903641
8,"HashingVectorizer(analyzer='char', n_features=...",0.901222
7,"HashingVectorizer(analyzer='char', n_features=...",0.897196
6,"HashingVectorizer(analyzer='char', n_features=...",0.895273
2,"HashingVectorizer(analyzer='char', n_features=...",0.882224
4,"TfidfVectorizer(ngram_range=(1, 3))",0.666330
3,"CountVectorizer(ngram_range=(1, 3))",0.649712
13,"HashingVectorizer(n_features=10000, ngram_rang...",0.620867


In [33]:
result_model_acc = pd.DataFrame({'Vectorizer': vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word,
                            'Accuracy': accuracy_scores})
result_model_acc.sort_values('Accuracy', ascending=False)

,Vectorizer,Accuracy
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.929504
1,"TfidfVectorizer(analyzer='char', ngram_range=(...",0.923583
9,"HashingVectorizer(analyzer='char', n_features=...",0.907417
8,"HashingVectorizer(analyzer='char', n_features=...",0.905000
7,"HashingVectorizer(analyzer='char', n_features=...",0.900835
6,"HashingVectorizer(analyzer='char', n_features=...",0.898939
2,"HashingVectorizer(analyzer='char', n_features=...",0.885165
4,"TfidfVectorizer(ngram_range=(1, 3))",0.656643
3,"CountVectorizer(ngram_range=(1, 3))",0.641139
13,"HashingVectorizer(n_features=10000, ngram_rang...",0.630722


In [34]:
result.sort_values('Accuracy', ascending=False).head(1)

,Tagger,Accuracy
6,UnigramTagger+BigramTagger+TrigramTagger,0.827905


In [35]:
result_model.sort_values('f1_score', ascending=False).head(1)

,Vectorizer,f1_score
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.927826


In [36]:
result_model_acc.sort_values('Accuracy', ascending=False).head(1)


,Vectorizer,Accuracy
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.929504


### Задание 2. Проверить насколько хорошо работает NER
данные брать из http://www.labinform.ru/pub/named_entities/
проверить NER из nltk/spacy/deeppavlov
написать свой нер попробовать разные подходы
передаём в сетку токен и его соседей
передаём в сетку только токен
свой вариант
сравнить ваши реализованные подходы на качество (вывести precision/recall/f1_score)


In [40]:
!unzip /content/collection3.zip

Archive:  /content/collection3.zip
   creating: Collection3/
  inflating: Collection3/001.ann     
  inflating: Collection3/001.txt     
  inflating: Collection3/002.ann     
  inflating: Collection3/002.txt     
  inflating: Collection3/003.ann     
  inflating: Collection3/003.txt     
  inflating: Collection3/004.ann     
  inflating: Collection3/004.txt     
  inflating: Collection3/005.ann     
  inflating: Collection3/005.txt     
  inflating: Collection3/006.ann     
  inflating: Collection3/006.txt     
  inflating: Collection3/007.ann     
  inflating: Collection3/007.txt     
  inflating: Collection3/008.ann     
  inflating: Collection3/008.txt     
  inflating: Collection3/009.ann     
  inflating: Collection3/009.txt     
  inflating: Collection3/010.ann     
  inflating: Collection3/010.txt     
  inflating: Collection3/011.ann     
  inflating: Collection3/011.txt     
  inflating: Collection3/012.ann     
  inflating: Collection3/012.txt     
  inflating: Collection3/01

In [41]:
print(os.listdir("/content/Collection3"))

['233.txt', '390.txt', '378.txt', '2001.ann', '370.ann', '293.ann', '389.txt', '261.ann', '063.txt', '506.txt', '453.ann', '1144.ann', '273.ann', '186.ann', '302.txt', '562.ann', 'last_19.ann', 'last_49.txt', '526.ann', '389.ann', 'last_24.ann', '096.ann', '297.ann', '003.txt', '299.ann', '137.ann', 'artjakov.txt', '587.ann', '2046.txt', '03_12_12d.txt', '145.ann', '317.txt', '355.ann', '447.txt', '398.ann', '234.txt', '510.txt', '126.txt', '1124.ann', '152.ann', '1122.ann', '186.txt', '1113.txt', '619.ann', 'last_08.ann', '1040.ann', '057.ann', '043.ann', '03_12_12a.txt', '062.txt', '400.txt', '1021.ann', '633.ann', '09_01_13e.ann', 'last_11.txt', '038.ann', '572.txt', 'mvd2.txt', '515.ann', '1121.txt', '031.ann', '370.txt', '23_11_12d.ann', '425.ann', '1102.txt', '458.ann', '2046.ann', '1148.ann', '058.txt', '486.txt', '419.txt', '249.ann', '1180.txt', '179.txt', '1142.ann', '530.txt', '574.ann', '183.txt', 'last_51.ann', '1100.ann', '358.txt', '028.txt', '557.txt', '14_01_13i.ann', 

In [43]:
fileDir = r"/content/Collection3"
fileExt = r".txt"
documents_txt = [_ for _ in os.listdir(fileDir) if _.endswith(fileExt)]
print(documents_txt)

['233.txt', '390.txt', '378.txt', '389.txt', '063.txt', '506.txt', '302.txt', 'last_49.txt', '003.txt', 'artjakov.txt', '2046.txt', '03_12_12d.txt', '317.txt', '447.txt', '234.txt', '510.txt', '126.txt', '186.txt', '1113.txt', '03_12_12a.txt', '062.txt', '400.txt', 'last_11.txt', '572.txt', 'mvd2.txt', '1121.txt', '370.txt', '1102.txt', '058.txt', '486.txt', '419.txt', '1180.txt', '179.txt', '530.txt', '183.txt', '358.txt', '028.txt', '557.txt', '345.txt', '134.txt', '220.txt', '583.txt', 'shojgu3.txt', '23_11_12c.txt', '479.txt', '595.txt', '1011.txt', '1175.txt', 'last_60.txt', '206.txt', '04_12_12h_corr.txt', '230.txt', '131.txt', '022.txt', 'blokhin.txt', '166.txt', 'last_58.txt', '1165.txt', '262.txt', '167.txt', '019.txt', '161.txt', '117.txt', '2047.txt', 'last_43.txt', '148.txt', '2010.txt', '380.txt', '1048.txt', '1016.txt', '033.txt', '079.txt', 'last_46.txt', '513.txt', '617.txt', '147.txt', 'last_66.txt', '1029.txt', '056.txt', '505.txt', '28_11_12g.txt', '250.txt', '1046.t

In [45]:
text_list = []
for file in documents_txt:
    doc = open('/content/Collection3/' + file, encoding='utf-8')
    text = doc.read()
    text_list.append(text)
    
data_text = pd.DataFrame({'text': text_list })
data_text

,text
0,Президент России уволил 6 генералов МВД\n \nПр...
1,"Банк России лишил лицензии ""Евросоюз""\n\nБанк ..."
2,Убит глава Сергиева Посада\n\nВ Московской обл...
3,В Абхазии начались президентские выборы\n\nВ А...
4,В Минобороны России проведены масштабные перес...
...,...
995,"""Партия пенсионеров России"" выдвинет кандидата..."
996,Президент Алжира провел перестановки в кабинет...
997,Вице-губернатор Санкт-Петербурга подала в отст...
998,Д.Медведев отправил в отставку губернатора Ста...


In [46]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [47]:
document = data_text.text[0]

nltk.pos_tag(nltk.word_tokenize(document))


[('Президент', 'JJ'),
 ('России', 'NNP'),
 ('уволил', 'NNP'),
 ('6', 'CD'),
 ('генералов', 'NNP'),
 ('МВД', 'NNP'),
 ('Президент', 'NNP'),
 ('России', 'NNP'),
 ('Дмитрий', 'NNP'),
 ('Медведев', 'NNP'),
 ('продолжает', 'NNP'),
 ('перестановки', 'NNP'),
 ('в', 'NNP'),
 ('руководстве', 'NNP'),
 ('МВД', 'NNP'),
 ('.', '.'),
 ('Сегодня', 'VB'),
 ('он', 'JJ'),
 ('уволил', '$'),
 ('6', 'CD'),
 ('генералов', 'NNP'),
 ('милиции', 'NNP'),
 ('.', '.'),
 ('В', 'NN'),
 ('частности', 'NN'),
 (',', ','),
 ('указом', 'NNP'),
 ('президента', 'NNP'),
 ('отправлен', 'NNP'),
 ('в', 'NNP'),
 ('отставку', 'NNP'),
 ('начальник', 'NNP'),
 ('Центра', 'NNP'),
 ('обеспечения', 'NNP'),
 ('оперативно-служебной', 'JJ'),
 ('деятельности', 'NNP'),
 ('по', 'NNP'),
 ('противодействию', 'NNP'),
 ('экстремизму', 'NNP'),
 ('МВД', 'NNP'),
 ('генерал-майор', 'JJ'),
 ('милиции', 'NNP'),
 ('Владимир', 'NNP'),
 ('Булатов', 'NNP'),
 ('.', '.'),
 ('Не', 'VB'),
 ('взяли', 'JJ'),
 ('на', 'NNP'),
 ('работу', 'NNP'),
 ('в', 'NNP'),


In [49]:
import nltk
nltk.download('maxent_ne_chunker')
nltk.download('words')


[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [50]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

{('Николая Михайлова', 'PERSON'),
 ('Президент', 'PERSON'),
 ('России', 'ORGANIZATION'),
 ('России Дмитрий Медведев', 'PERSON'),
 ('Сибирского', 'ORGANIZATION'),
 ('Управления', 'ORGANIZATION'),
 ('Центра', 'PERSON')}

In [52]:
pd.read_csv('/content/Collection3/001.ann', delimiter='\t')

,T1,LOC 0 6,Россия
0,T2,LOC 50 53,США
1,T3,LOC 57 63,Грузию
2,T4,LOC 87 93,МОСКВА
3,T5,ORG 103 114,РИА Новости
4,T6,LOC 116 122,Россия
5,T7,LOC 141 144,США
6,T8,LOC 161 168,Тбилиси
7,T9,LOC 301 307,России
8,T10,PER 308 324,Григорий Карасин
9,T11,LOC 383 386,США


In [53]:
nlp = ru_core_news_sm.load()
ny_bb = data_text.text[0]
article = nlp(ny_bb)
displacy.render(article, jupyter=True, style='ent')

In [54]:
for token in article:
    print(token.text, token.pos_, token.dep_)

Президент NOUN nsubj
России PROPN nmod
уволил VERB ROOT
6 NUM nummod
генералов NOUN obj
МВД PROPN nmod

 
 SPACE dep
Президент NOUN nsubj
России PROPN nmod
Дмитрий PROPN appos
Медведев PROPN flat:name
продолжает VERB conj
перестановки NOUN obj
в ADP case
руководстве NOUN nmod
МВД PROPN nmod
. PUNCT punct
Сегодня ADV advmod
он PRON nsubj
уволил VERB ROOT
6 NUM nummod
генералов NOUN obj
милиции NOUN nmod
. PUNCT punct


 SPACE dep
В ADP discourse
частности NOUN fixed
, PUNCT punct
указом NOUN obl
президента NOUN nmod
отправлен VERB ROOT
в ADP case
отставку NOUN obl
начальник NOUN nsubj:pass
Центра PROPN nmod
обеспечения NOUN nmod
оперативно ADJ amod
- ADJ amod
служебной ADJ amod
деятельности NOUN nmod
по ADP case
противодействию NOUN nmod
экстремизму NOUN iobj
МВД PROPN nmod
генерал NOUN appos
- NOUN appos
майор NOUN appos
милиции NOUN nmod
Владимир PROPN appos
Булатов PROPN flat:name
. PUNCT punct
Не PART advmod
взяли VERB ROOT
на ADP case
работу NOUN obl
в ADP case
полицию NOUN obl
и C

In [56]:
pd.read_csv('/content/Collection3/001.ann', delimiter='\t', header=None)

,0,1,2
0,T1,LOC 0 6,Россия
1,T2,LOC 50 53,США
2,T3,LOC 57 63,Грузию
3,T4,LOC 87 93,МОСКВА
4,T5,ORG 103 114,РИА Новости
5,T6,LOC 116 122,Россия
6,T7,LOC 141 144,США
7,T8,LOC 161 168,Тбилиси
8,T9,LOC 301 307,России
9,T10,PER 308 324,Григорий Карасин


In [57]:
!pip install natasha

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 34.4 MB 128 kB/s 
     |████████████████████████████████| 41 kB 36 kB/s 


In [58]:
!pip install navec


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [60]:
text = data_text.text[0]
navec = Navec.load('/content/navec_news_v1_1B_250K_300d_100q.tar')
ner = NER.load('/content/slovnet_ner_news_v1.tar')
ner.navec(navec)

markup = ner(text)
show_markup(markup.text, markup.spans)

Президент России уволил 6 генералов МВД
          LOC───                    ORG
 
Президент России Дмитрий Медведев продолжает перестановки в 
          LOC─── PER─────────────                           
руководстве МВД. Сегодня он уволил 6 генералов милиции.
            ORG                                        
В частности, указом президента отправлен в отставку начальник Центра 
                                                              ORG────
обеспечения оперативно-служебной деятельности по противодействию 
─────────────────────────────────────────────                    
экстремизму МВД генерал-майор милиции Владимир Булатов. Не взяли на 
            ORG                       PER─────────────              
работу в полицию и ряд региональных руководителей ведомства. Уволены 
начальник Управления на транспорте МВД по Северо-Кавказскому 
          ORG───────────────────── ORG    LOC────────────────
федеральному округу генерал-майор милиции Игорь Жуков, начальник 
──────────────

In [61]:
fileDir = r"/content/Collection3"
fileExt = r".ann"
documents_ann = [_ for _ in os.listdir(fileDir) if _.endswith(fileExt)]
print(documents_ann)

['2001.ann', '370.ann', '293.ann', '261.ann', '453.ann', '1144.ann', '273.ann', '186.ann', '562.ann', 'last_19.ann', '526.ann', '389.ann', 'last_24.ann', '096.ann', '297.ann', '299.ann', '137.ann', '587.ann', '145.ann', '355.ann', '398.ann', '1124.ann', '152.ann', '1122.ann', '619.ann', 'last_08.ann', '1040.ann', '057.ann', '043.ann', '1021.ann', '633.ann', '09_01_13e.ann', '038.ann', '515.ann', '031.ann', '23_11_12d.ann', '425.ann', '458.ann', '2046.ann', '1148.ann', '249.ann', '1142.ann', '574.ann', 'last_51.ann', '1100.ann', '14_01_13i.ann', '1014.ann', 'turkmenija.ann', '259.ann', '1015.ann', 'last_44.ann', '509.ann', 'last_16.ann', '090.ann', 'last_52.ann', '25_12_12d.ann', 'last_18.ann', '2004.ann', '20_11_12d.ann', '330.ann', '1042.ann', '427.ann', '2039.ann', '1009.ann', '1153.ann', '107.ann', '1001.ann', '140.ann', '15_01_13a.ann', 'last_01.ann', '1003.ann', '189.ann', '09_01_13c.ann', '182.ann', '25_12_12a.ann', '289.ann', '498.ann', '1047.ann', '356.ann', '017.ann', '591.ann

In [62]:
ann = pd.read_csv('/content/Collection3/001.ann', delimiter='\t', header=None)
ann

,0,1,2
0,T1,LOC 0 6,Россия
1,T2,LOC 50 53,США
2,T3,LOC 57 63,Грузию
3,T4,LOC 87 93,МОСКВА
4,T5,ORG 103 114,РИА Новости
5,T6,LOC 116 122,Россия
6,T7,LOC 141 144,США
7,T8,LOC 161 168,Тбилиси
8,T9,LOC 301 307,России
9,T10,PER 308 324,Григорий Карасин


In [63]:
import csv

docs = []
for i in range(len(data_text)):
    words = []
    labels = []
    text = data_text['text'][i]
    
    df = pd.read_csv('/content/Collection3/' + documents_ann[i], delimiter='\t', header=None, quoting=csv.QUOTE_NONE, encoding='utf-8')
    df_ann = pd.DataFrame()
    df_ann['Token'] = df.loc[:, 2]
    split_1 = [loc.split() for loc in df.loc[:, 1].values]
    df_ann['Entity'] = [loc[0] for loc in split_1]
       
    dic = {}
    for j in range(len(df)):
        token = df_ann['Token'][j].lower().split()
        entity = df_ann['Entity'][j]
        for tok in token:
            dic[tok] = entity

    for token in tokenize(text):
        if (token.text.lower() in dic.keys()):
            words.append(token.text)
            labels.append(dic[token.text.lower()])
        else:
            words.append(token.text)
            labels.append('OUT')
    
    docs.append([words, labels])

In [64]:
print(docs[0][0]), print(docs[0][1])

['Президент', 'России', 'уволил', '6', 'генералов', 'МВД', 'Президент', 'России', 'Дмитрий', 'Медведев', 'продолжает', 'перестановки', 'в', 'руководстве', 'МВД', '.', 'Сегодня', 'он', 'уволил', '6', 'генералов', 'милиции', '.', 'В', 'частности', ',', 'указом', 'президента', 'отправлен', 'в', 'отставку', 'начальник', 'Центра', 'обеспечения', 'оперативно-служебной', 'деятельности', 'по', 'противодействию', 'экстремизму', 'МВД', 'генерал-майор', 'милиции', 'Владимир', 'Булатов', '.', 'Не', 'взяли', 'на', 'работу', 'в', 'полицию', 'и', 'ряд', 'региональных', 'руководителей', 'ведомства', '.', 'Уволены', 'начальник', 'Управления', 'на', 'транспорте', 'МВД', 'по', 'Северо-Кавказскому', 'федеральному', 'округу', 'генерал-майор', 'милиции', 'Игорь', 'Жуков', ',', 'начальник', 'Управления', 'внутренних', 'дел', 'по', 'Тверской', 'области', 'генерал-майор', 'милиции', 'Александр', 'Куликов', ',', 'начальник', 'Управления', 'внутренних', 'дел', 'по', 'Тамбовской', 'области', 'генерал-майор', 'мил

(None, None)

In [65]:
data, labels = list(zip(*docs))
for w, e in zip(data[0], labels[0]):
    print(f'{w}\t{e}')

Президент	OUT
России	OUT
уволил	OUT
6	OUT
генералов	OUT
МВД	OUT
Президент	OUT
России	OUT
Дмитрий	OUT
Медведев	OUT
продолжает	OUT
перестановки	OUT
в	OUT
руководстве	OUT
МВД	OUT
.	OUT
Сегодня	OUT
он	OUT
уволил	OUT
6	OUT
генералов	OUT
милиции	OUT
.	OUT
В	OUT
частности	OUT
,	OUT
указом	OUT
президента	OUT
отправлен	OUT
в	OUT
отставку	OUT
начальник	OUT
Центра	OUT
обеспечения	OUT
оперативно-служебной	OUT
деятельности	OUT
по	OUT
противодействию	OUT
экстремизму	OUT
МВД	OUT
генерал-майор	OUT
милиции	OUT
Владимир	OUT
Булатов	OUT
.	OUT
Не	OUT
взяли	OUT
на	OUT
работу	OUT
в	OUT
полицию	OUT
и	OUT
ряд	OUT
региональных	OUT
руководителей	OUT
ведомства	OUT
.	OUT
Уволены	OUT
начальник	OUT
Управления	OUT
на	OUT
транспорте	OUT
МВД	OUT
по	OUT
Северо-Кавказскому	OUT
федеральному	OUT
округу	OUT
генерал-майор	OUT
милиции	OUT
Игорь	OUT
Жуков	OUT
,	OUT
начальник	OUT
Управления	OUT
внутренних	OUT
дел	OUT
по	OUT
Тверской	OUT
области	OUT
генерал-майор	OUT
милиции	OUT
Александр	OUT
Куликов	OUT
,	OUT
начальник	OUT
Упр

In [66]:
df = pd.DataFrame({'sent_id': [i for j in [[i] * len(s) for i, s in enumerate(data)] for i in j],
                   'data': [i for j in data for i in j],
                   'entities': [i for j in labels for i in j]})
df.head(50)

,sent_id,data,entities
0,0,Президент,OUT
1,0,России,OUT
2,0,уволил,OUT
3,0,6,OUT
4,0,генералов,OUT
5,0,МВД,OUT
6,0,Президент,OUT
7,0,России,OUT
8,0,Дмитрий,OUT
9,0,Медведев,OUT


In [67]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s['data'].values.tolist(), 
                                                           s['entities'].values.tolist())]
        self.grouped = self.data.groupby('sent_id').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

getter = SentenceGetter(df)

sentences = getter.sentences


In [68]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isdigit()': word.isdigit()
    }
    if i > 0:
        word1 = sent[i - 1][0]
        features.update({
            '-1:word.lower()': word1.lower()
        })
    else:
        features['BOS'] = True
    if i < len(sent) - 1:
        word1 = sent[i + 1][0]
        features.update({
            '+1:word.lower()': word1.lower()
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]


In [69]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [70]:
X[2][7]

{'+1:word.lower()': 'глава',
 '-1:word.lower()': 'области',
 'bias': 1.0,
 'word.isdigit()': False,
 'word.lower()': 'убит',
 'word[-2:]': 'ит',
 'word[-3:]': 'бит'}

In [71]:
len(X)

1000

In [72]:
X_train = X[:700]
X_test = X[700:]
y_train = y[:700]
y_test = y[700:]

In [73]:
%%time

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', #Градиентный спуск с использованием метода L-BFGS
    c1=0.1, #Коэффициент для регуляризации L1
    c2=0.1, #Коэффициент для регуляризации L2
    max_iterations=1000, #Максимальное количество итераций
    all_possible_transitions=True, #Генерация объектов (не встречающихся в обучающих данных)
    verbose=True #Включение режима тренировки
)

try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass

loading training data to CRFsuite: 100%|██████████| 700/700 [00:01<00:00, 672.54it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 86064
Seconds required: 0.268

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 1000
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.14  loss=64659.70 active=85984 feature_norm=1.00
Iter 2   time=0.08  loss=12521.72 active=85511 feature_norm=2.09
Iter 3   time=0.08  loss=12168.51 active=85777 feature_norm=2.15
Iter 4   time=0.07  loss=11509.93 active=41303 feature_norm=2.35
Iter 5   time=0.08  loss=11332.90 active=28913 feature_norm=2.45
Iter 6   time=0.09  loss=11096.45 active=21466 feature_norm=2.59
Iter 7   time=0.07  loss=10514.33 active=15621 feature_norm=2.79
Iter 8   time=0.08  loss=9973.67  active=12151 feature_norm=4.52
Iter 9   time=0.08  loss=8854.75  active=14774 feature_norm=4.28
Iter 10  time

In [74]:
all_entities = sorted(df.entities.unique().tolist())
len(all_entities)

4

In [75]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=all_entities)


0.9862427948930688

In [76]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isdigit()': word.isdigit()
    }
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [77]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [78]:
X[1][7]

{'bias': 1.0,
 'word.isdigit()': False,
 'word.lower()': 'банк',
 'word[-2:]': 'нк',
 'word[-3:]': 'анк'}

In [79]:
len(X)

1000

In [80]:
X_train = X[:700]
X_test = X[700:]
y_train = y[:700]
y_test = y[700:]

In [81]:
%%time

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=1000,
    all_possible_transitions=True,
    verbose=True
)

try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass

loading training data to CRFsuite: 100%|██████████| 700/700 [00:00<00:00, 873.82it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 32210
Seconds required: 0.151

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 1000
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.12  loss=65055.53 active=32208 feature_norm=1.00
Iter 2   time=0.06  loss=12457.04 active=32142 feature_norm=2.10
Iter 3   time=0.07  loss=12134.31 active=32157 feature_norm=2.16
Iter 4   time=0.06  loss=11510.75 active=16195 feature_norm=2.36
Iter 5   time=0.06  loss=11341.96 active=11544 feature_norm=2.46
Iter 6   time=0.07  loss=11106.80 active=8327  feature_norm=2.60
Iter 7   time=0.06  loss=10528.35 active=6256  feature_norm=2.80
Iter 8   time=0.08  loss=10332.54 active=5053  feature_norm=4.38
Iter 9   time=0.07  loss=8986.39  active=5403  feature_norm=4.12
Iter 10  time

In [82]:
all_entities = sorted(df.entities.unique().tolist())
len(all_entities)

4

In [83]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=all_entities)

0.9862304531240356